In [ ]:
!pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


**IMporting Tweeter Sentimental Dataset**

In [ ]:
#API to fix dataset from Kaggle

!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:03<00:00, 24.5MB/s]
100% 80.9M/80.9M [00:03<00:00, 21.6MB/s]


In [ ]:
#extracting ZIP File

from zipfile import ZipFile
Dataset = "sentiment140.zip"

with ZipFile(Dataset, 'r') as zip:
  zip.extractall()
  print("Done")

Done


***IMporting The Dependencies ***

In [ ]:
import numpy as np
import pandas as pd
import re
#re = regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing stopwords in english
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Processing**

In [ ]:
#loading the data from dataset
tweets=pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1 ")

In [ ]:
tweets.shape

(1599999, 6)

In [ ]:
tweets.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the columns and reading the dataset again

column_names = ["target", "ids", "date", "flag", "user", "tweet"]
tweets = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", names = column_names)


In [ ]:
tweets.shape


(1600000, 6)

In [ ]:
tweets.head()

,target,ids,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting missing values
tweets.isnull().sum()


target    0
ids       0
date      0
flag      0
user      0
tweet     0
dtype: int64

In [ ]:
#checking the distributions
tweets['target'].value_counts()


target
0    800000
4    800000
Name: count, dtype: int64

**Converting the target  4 to 1**

In [ ]:
tweets.replace({'target':{4:1}},inplace=True)

In [ ]:
tweets['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0-> Negative
1-> Positive

**Stemming**

process of reducing a word to its root word
ex= actor , actress, acting = act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content
#

In [ ]:
tweets['stemmed_content'] = tweets['tweet'].apply(stemming)

In [ ]:
tweets.head()

,target,ids,date,flag,user,tweet,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(tweets['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(tweets['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#separating the data and label
X= tweets['stemmed_content'].values
Y= tweets['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
print(X_train)

  (0, 8)	0.408248290463863
  (0, 4)	0.408248290463863
  (0, 1)	0.408248290463863
  (0, 3)	0.408248290463863
  (0, 5)	0.408248290463863
  (0, 7)	0.408248290463863
  (1, 2)	0.5773502691896257
  (1, 6)	0.5773502691896257
  (1, 0)	0.5773502691896257


In [ ]:
print(X_test)

  (39, 7)	1.0
  (71, 1)	1.0
  (76, 7)	1.0
  (96, 1)	1.0
  (101, 5)	1.0
  (191, 7)	1.0
  (200, 6)	1.0
  (265, 7)	1.0
  (270, 7)	1.0
  (294, 5)	1.0
  (316, 7)	1.0
  (324, 7)	1.0
  (350, 5)	1.0
  (358, 7)	1.0
  (361, 7)	1.0
  (371, 5)	1.0
  (382, 5)	1.0
  (385, 7)	1.0
  (428, 5)	1.0
  (432, 7)	1.0
  (436, 7)	1.0
  (464, 5)	1.0
  (470, 7)	1.0
  (498, 5)	1.0
  (527, 1)	1.0
  :	:
  (319393, 1)	1.0
  (319453, 5)	1.0
  (319459, 7)	1.0
  (319488, 7)	1.0
  (319518, 4)	1.0
  (319543, 5)	1.0
  (319556, 7)	1.0
  (319560, 7)	1.0
  (319592, 5)	1.0
  (319597, 6)	1.0
  (319608, 6)	1.0
  (319681, 7)	1.0
  (319700, 7)	1.0
  (319747, 8)	0.5773502691896257
  (319747, 7)	0.5773502691896257
  (319747, 1)	0.5773502691896257
  (319769, 7)	1.0
  (319777, 1)	1.0
  (319862, 1)	1.0
  (319865, 5)	1.0
  (319921, 1)	1.0
  (319970, 5)	1.0
  (319974, 1)	1.0
  (319977, 7)	1.0
  (319984, 7)	1.0


**Training The ML Model**

In [ ]:
model= LogisticRegression(max_iter=1000)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_vectorized = vectorizer.transform(X_test)

# Now use these vectorized versions in your model
model.fit(X_train_vectorized, Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [ ]:
#score on the training data
X_train_prediction = model.predict(X_train_vectorized)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy on training data:",training_data_accuracy  )

Accuracy on training data: 0.809959375


In [ ]:
X_test_prediction = model.predict(X_test_vectorized)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("Accuracy on training data:",test_data_accuracy  )

Accuracy on training data: 0.778215625


**Model Accuracy 77.8%**

**Saving the Trained Model**

In [ ]:
import  pickle as pkl

In [ ]:
filename="model.sav"
pkl.dump(model,open(filename,'wb'))

**Using the saved model for future work'**

In [ ]:
loaded_model= pkl.load(open('model.sav','rb'))

In [ ]:
X_new = X_test[200]
print( Y_test[200])

# Assuming you used a vectorizer (like TfidfVectorizer) during training
X_new_vectorized = vectorizer.transform([X_new])  # Note the list brackets

prediction = loaded_model.predict(X_new_vectorized)
print("Prediction:", prediction)

if prediction[0] == 0:
    print("Negative")
else:
    print("Positive")

0
Prediction: [0]
Negative


In [ ]:
X_new = X_test[1]
print( Y_test[1])

# Assuming you used a vectorizer (like TfidfVectorizer) during training
X_new_vectorized = vectorizer.transform([X_new])  # Note the list brackets

prediction = loaded_model.predict(X_new_vectorized)
print("Prediction:", prediction)

if prediction[0] == 0:
    print("Negative")
else:
    print("Positive")

0
Prediction: [1]
Positive
